# Importing required libraries

pip install -r requirements.txt

In [29]:
# pip install langchain langchain-core langchain-community Ollama
# !pip install -U langchain-experimental
# !pip install pymysql
# !pip install -U langchain-ollama


In [ ]:
# LangChain + SQL + LLM setup
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import PromptTemplate
# from langchain_experimental.sql.base import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# LLM options
# from langchain.llms import OpenAI
from langchain_ollama import OllamaLLM
# from langchain_google_genai import ChatGoogleGenerativeAI

- from langchain.utilities import SQLDatabase → Connects LangChain to SQL databases for querying and schema access.
- from langchain.prompts import PromptTemplate → Creates structured prompts with placeholders for dynamic input to LLMs.
- from langchain.chains import create_sql_query_chain → Builds a chain that converts natural language into SQL and executes it.
- from langchain_core.output_parsers import StrOutputParser → Parses LLM output as plain strings for clean result handling.
- from langchain_core.runnables import RunnablePassthrough → Passes input directly to output without modification, useful in chains.
- from langchain.llms import Ollama → Loads and runs local LLMs like LLaMA or Mistral via Ollama integration.

since "from langchain_experimental.sql.base import create_sql_query_chain" did not work, we have to manually write prompt

 connect to MySql

In [2]:
host = "localhost"
port = 3306
username = "root"
database_schema = "text_to_sql"

#since our password contain special character
from urllib.parse import quote_plus
password = quote_plus("Shreyash@6105")

mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=2)


In [3]:
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=2)
db.get_table_info()

'\nCREATE TABLE `2017_budgets` (\n\t`Product Name` TEXT, \n\t`2017 Budgets` DOUBLE\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n2 rows from 2017_budgets table:\nProduct Name\t2017 Budgets\nProduct 1\t3016489.2089999998\nProduct 2\t3050087.5649999999\n*/\n\n\nCREATE TABLE customers (\n\t`Customer Index` INTEGER, \n\t`Customer Names` TEXT\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n2 rows from customers table:\nCustomer Index\tCustomer Names\n1\tGeiss Company\n2\tJaxbean Group\n*/\n\n\nCREATE TABLE products (\n\t`Index` INTEGER, \n\t`Product Name` TEXT\n)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci\n\n/*\n2 rows from products table:\nIndex\tProduct Name\n1\tProduct 1\n2\tProduct 2\n*/\n\n\nCREATE TABLE regions (\n\tid INTEGER, \n\tname TEXT, \n\tcounty TEXT, \n\tstate_code TEXT, \n\tstate TEXT, \n\ttype TEXT, \n\tlatitude DOUBLE, \n\tlongitude DOUBLE, \n\tarea_code INTEGER, \n\tpopulation INTEGER, \n\thousehol

In [10]:
prompt = PromptTemplate.from_template("""
You are a MySQL expert.

Generate a syntactically correct MySQL SQL query for the given question.

CRITICAL RULES (MUST FOLLOW):
- Use ONLY table and column names EXACTLY as they appear in the schema
- Column names may contain SPACES — preserve them exactly
- ALWAYS wrap column names containing spaces in BACKTICKS (`column name`)
- Do NOT convert names to snake_case
- Generate READ-ONLY queries (SELECT only)
- Do NOT hallucinate tables or columns
- Do NOT add explanations or formatting
- Return ONLY the SQL query

Schema:
{schema}

Question:
{question}

SQL Query:
""")

In [11]:
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [12]:
llm = OllamaLLM(
    model="llama3",
    temperature=0
)

Create the SQL query chain using the llm and the prompt template

In [13]:
sql_chain = (
    RunnablePassthrough.assign(
        schema=lambda _: db.get_table_info()
    )
    | prompt
    | llm
    | StrOutputParser()
)


User Input
   ↓
Add DB schema
   ↓
Prompt template
   ↓
LLM (Ollama)
   ↓
String Output


### test SQL Query Chain with a sample question

In [14]:
response = sql_chain.invoke({"question":"What is the total 'Line Total' for Geiss Company"})
print(response)

SELECT SUM(`Line Total`)
FROM `sales_order`
WHERE `Customer Name Index` = (SELECT `Customer Index` FROM `customers` WHERE `Customer Names` = 'Geiss Company');


In [15]:
db.run(response)

'[(5516846.999999994,)]'

#
#
#
#
### RAGAS implementation

### it is for further Alterations in this project 